In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pprint import pprint
from pathlib import Path
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single
import bids
from bids import BIDSLayout

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Enter the path to the Kamitani dataset, example: X:\\Datasets\\Deep-Image-Reconstruction\\
dataset_path = Path(input())

 X:\Datasets\Deep-Image-Reconstruction\derivatives\fmriprep-20.2.4


In [4]:
# Load layouts, this will take about a minute because pybids is slow

t = time.time()
dataset_layout = bids.BIDSLayout(dataset_path)
print(time.time() - t)

74.23182034492493


In [5]:
# There were 15 sessions to acquire training data in this dataset
# These three groups of sessions presented the same images in the same runs
# Each image in the training set was presented exactly 5 times
# It is better to run GLMsingle on the same run from one of these groups so cross validation is possible
session_groups = {
    'A': [f'perceptionNaturalImageTraining{i:02}' for i in (1, 4, 7, 10, 13)],
    'B': [f'perceptionNaturalImageTraining{i:02}' for i in (2, 5, 8, 11, 14)],
    'C': [f'perceptionNaturalImageTraining{i:02}' for i in (3, 6, 9, 12, 15)],
}

WindowsPath('sub-02/ses-perceptionNaturalImageTraining13/func/sub-02_ses-perceptionNaturalImageTraining13_task-perception_run-1_space-T1w_desc-preproc_bold.nii.gz')

In [15]:
import zipfile

sessions = session_groups['A'] # select one or more session groups
subject = '02' # ['01', '02', '03']
space = 'T1w'
run_ids = [1] # one or more runs

with zipfile.ZipFile(dataset_path / 'kamitani.zip', 'w', zipfile.ZIP_DEFLATED) as f:
    
    f.write(dataset_path / 'dataset_description.json', '.')
    for session in sessions:
        for run_id in run_ids:

            bids_image = dataset_layout.get(
                subject=subject, 
                session=session, 
                space=space, 
                run=run_id, 
                desc='preproc', 
                extension='nii.gz'
            )[0]
            events_file = dataset_layout.get(
                subject=subject, 
                session=session, 
                run=run_id, 
                extension='tsv', 
            )[0]
            
            f.write(bids_image.path, str(Path(bids_image.path).relative_to(dataset_path)))
            f.write(events_file.path, str(Path(events_file.path).relative_to(dataset_path)))

In [7]:
sessions = session_groups['A'] # select one or more session groups
subject = '02' # ['01', '02', '03']
space = 'T1w'
run_ids = [1] # one or more runs

run_images = []
events_dfs = []
for session in sessions:
    for run_id in run_ids:

        bids_image = derivatives_layout.get(
            subject=subject, 
            session=session, 
            space=space, 
            run=run_id, 
            desc='preproc', 
            extension='nii.gz'
        )[0]

        events_files = dataset_layout.get(
            subject=subject, 
            session=session, 
            run=run_id, 
            extension='tsv', 
        )
        events_df = pd.read_csv(events_files[0].path, sep='\t')

        events_dfs.append(events_df)
        run_images.append(bids_image.get_image())

In [96]:
mask_image = derivatives_layout.get(
    subject=subject, 
    session=session, 
    space=space, 
    run=run_id, 
    desc='brain', 
    extension='nii.gz'
)[0].get_image()
fmri_mask = mask_image.get_fdata().astype(bool)
H, W, D = fmri_mask.shape

fmri_batch = []
for run_image in tqdm(run_images):
    fmri_data = run_image.get_fdata()
    fmri_data = fmri_data[fmri_mask]
    fmri_batch.append(fmri_data)
    

In [98]:
# Cell for visualizing the fMRI runs

all_runs = np.concatenate(fmri_batch)
mean, std = all_runs.mean(), all_runs.std()

show_volume = np.zeros_like(fmri_mask, dtype=float)
_, T = fmri_batch[0].shape

@interact(run_id=(0, num_runs-1), d=(0, D-1), t=(0, T-1))
def show(run_id, d, t):
    show_volume[fmri_mask] = fmri_batch[run_id][:, t]
    plt.figure(figsize=(8, 8))
    plt.imshow(show_volume[:, :, d], cmap='gray', vmin=mean-2*std, vmax=mean+2*std)
    plt.show()

ValueError: not enough values to unpack (expected 4, got 2)

In [137]:
fmri_batch[0].shape

(190037, 239)

In [135]:
tr = 2.

conditions = []
for run_id, events_df in enumerate(events):
    for i, event in events_df.iterrows():
        if int(event['event_type']) != 1:
            continue
        conditions.append(event['stimulus_id'])
conditions = list(set(conditions))
conditions.sort()
conditions = {condition: i  for i, condition in enumerate(conditions)}
C = len(conditions)

T = fmri_batch[0].shape[-1]

design_batch = []
for run_id, events_df in enumerate(events):
    design_matrix = np.zeros(shape=(T, C))
    
    for i, event in events_df.iterrows():
        if int(event['event_type']) != 1:
            continue
        condition_name = event['stimulus_id']
        c = conditions[condition_name]
        print(event['onset'] / tr)
        t = round(event['onset'] / tr)
        design_matrix[t, c] = 1
    design_batch.append(design_matrix)
    
@interact(run_id=(0, len(design_batch)-1))
def show(run_id):
    design_matrix = design_batch[run_id]
    print(design_matrix.sum(axis=0))
    plt.figure(figsize=(12, 12))
    plt.imshow(design_matrix)

16.0
20.0
24.0
28.0
32.0
40.0
44.0
48.0
52.0
56.0
60.0
64.0
68.0
72.0
76.0
80.0
84.0
88.0
96.0
100.0
104.0
108.0
116.0
120.0
128.0
132.0
140.0
144.0
148.0
152.0
156.0
160.0
164.0
168.0
172.0
176.0
180.0
184.0
188.0
192.0
196.0
200.0
204.0
208.0
212.0
216.0
220.0
224.0
228.0
232.0
16.0
20.0
24.0
28.0
32.0
40.0
44.0
48.0
52.0
56.0
60.0
64.0
68.0
72.0
76.0
80.0
84.0
88.0
96.0
100.0
104.0
108.0
116.0
120.0
128.0
132.0
140.0
144.0
148.0
152.0
156.0
160.0
164.0
168.0
172.0
176.0
180.0
184.0
188.0
192.0
196.0
200.0
204.0
208.0
212.0
216.0
220.0
224.0
228.0
232.0
16.0
20.0
24.0
28.0
32.0
40.0
44.0
48.0
52.0
56.0
60.0
64.0
68.0
72.0
76.0
80.0
84.0
88.0
96.0
100.0
104.0
108.0
116.0
120.0
128.0
132.0
140.0
144.0
148.0
152.0
156.0
160.0
164.0
168.0
172.0
176.0
180.0
184.0
188.0
192.0
196.0
200.0
204.0
208.0
212.0
216.0
220.0
224.0
228.0
232.0
16.0
20.0
24.0
28.0
32.0
40.0
44.0
48.0
52.0
56.0
60.0
64.0
68.0
72.0
76.0
80.0
84.0
88.0
96.0
100.0
104.0
108.0
116.0
120.0
128.0
132.0
140.0
144.0
148.0
15

interactive(children=(IntSlider(value=2, description='run_id', max=4), Output()), _dom_classes=('widget-intera…

In [102]:
glmsingle_obj = GLM_single(dict(
    wantlibrary=1,
    wantglmdenoise=1,
    wantfracridge=1,
    wantfileoutputs=[1,1,1,1],
    wantmemoryoutputs=[1,1,1,1],
))

pprint(glmsingle_obj.params)

output_path = dataset_path / f'derivatives/glmsingle/sub-{subject}/ses-groupA/run1'
results_glmsingle = glmsingle_obj.fit(
    design=design_batch,
    data=fmri_batch,
    stimdur=8,
    tr=2.,
    outputdir=str(output_path),
)

{'R2thresh': 0,
 'brainR2': [],
 'brainexclude': False,
 'brainthresh': [99.0, 0.1],
 'chunklen': 50000,
 'extra_regressors': False,
 'fracs': array([1.  , 0.95, 0.9 , 0.85, 0.8 , 0.75, 0.7 , 0.65, 0.6 , 0.55, 0.5 ,
       0.45, 0.4 , 0.35, 0.3 , 0.25, 0.2 , 0.15, 0.1 , 0.05]),
 'hrffitmask': 1,
 'hrfmodel': 'optimise',
 'hrfthresh': 0.5,
 'lambda': 0,
 'n_boots': 100,
 'n_jobs': 1,
 'n_pcs': 10,
 'numforhrf': 50,
 'pcR2cutoff': [],
 'pcR2cutoffmask': 1,
 'pcstop': 1.05,
 'seed': 1651561016.0385394,
 'suppressoutput': 0,
 'wantautoscale': 1,
 'wantfileoutputs': [1, 1, 1, 1],
 'wantfracridge': 1,
 'wantglmdenoise': 1,
 'wanthdf5': 0,
 'wantlibrary': 1,
 'wantlss': 0,
 'wantmemoryoutputs': [1, 1, 1, 1],
 'wantparametric': 0,
 'wantpercentbold': 1}
*** FITTING TYPE-A MODEL (ONOFF) ***



G:\python_env\glm-single\lib\site-packages\glmsingle\utils\chunking.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(f)


fitting model...
done.

preparing output...
done.

computing model fits...
done.

computing R^2...
done.

computing SNR...
done.


*** Saving results to X:\Datasets\Deep-Image-Reconstruction\derivatives\glmsingle\sub-02\ses-groupA\run1\TYPEA_ONOFF.npy. ***

*** Setting brain R2 threshold to 0.18844279562300356 ***

*** FITTING TYPE-B MODEL (FITHRF) ***



chunks: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:12<00:00, 48.02s/it]



*** Saving results to X:\Datasets\Deep-Image-Reconstruction\derivatives\glmsingle\sub-02\ses-groupA\run1\TYPEB_FITHRF.npy. ***

*** DETERMINING GLMDENOISE REGRESSORS ***

*** CROSS-VALIDATING DIFFERENT NUMBERS OF REGRESSORS ***



...


*** FITTING TYPE-C MODEL (GLMDENOISE) ***



chunks: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:40<00:00, 10.17s/it]



*** Saving results to X:\Datasets\Deep-Image-Reconstruction\derivatives\glmsingle\sub-02\ses-groupA\run1\TYPEC_FITHRF_GLMDENOISE.npy. ***

*** FITTING TYPE-D MODEL (GLMDENOISE_RR) ***



chunks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [12:16<00:00, 184.06s/it]



*** Saving results to X:\Datasets\Deep-Image-Reconstruction\derivatives\glmsingle\sub-02\ses-groupA\run1\TYPED_FITHRF_GLMDENOISE_RR.npy. ***

*** All model types done ***

*** return model types in results ***



In [105]:
for type_name, results in results_glmsingle.items():
    print(type_name)
    for result in results:
        print(result)
    print()

typea
onoffR2
meanvol
betasmd

typeb
FitHRFR2
FitHRFR2run
HRFindex
HRFindexrun
R2
R2run
betasmd
meanvol

typec
HRFindex
HRFindexrun
glmbadness
pcvoxels
pcnum
xvaltrend
noisepool
pcregressors
betasmd
R2
R2run
meanvol

typed
HRFindex
HRFindexrun
glmbadness
pcvoxels
pcnum
xvaltrend
noisepool
pcregressors
betasmd
R2
R2run
rrbadness
FRACvalue
scaleoffset
meanvol



In [132]:

plot_fields = ['betasmd','R2','HRFindex','FRACvalue']
colormaps = ['RdBu_r','hot','jet','copper']
clims = [[-5,5],[0,85],[0,20],[0,1]]
final_results = results_glmsingle['typed']

@interact(d_layer=(0, 60))
def show(d_layer):
    plt.figure(figsize=(12, 8))

    def plot_result(i, field, cmap, clim):
        plt.subplot(2,2, i)
        x = final_results[field]

        if len(x.shape) == 2:
            x = np.nanmean(x, axis=-1)

        volume = np.zeros_like(mask_data, dtype=np.float32)
        volume[mask_data] = x

        plt.imshow(volume[:, :, d_layer], cmap=cmap, clim=clim)

        plt.colorbar()
        plt.title(field)
        plt.axis(False)


    plot_result(1, 'betasmd', 'RdBu_r', [-5, 5])
    plot_result(2, 'R2', 'hot', [0, 55])
    plot_result(3, 'HRFindex', 'jet', [0, 20])
    plot_result(4, 'FRACvalue', 'copper', [0, 1])


interactive(children=(IntSlider(value=30, description='d_layer', max=60), Output()), _dom_classes=('widget-int…

In [56]:
type_names = {
    'typea': 'onoff',
    'typeb': 'fithrf',
    'typec': 'fithrf_GLMdenoise',
    'typed': 'fithrf_GLMdenoise_RR'
}

num_voxels = fmri_batch[0].shape[0]

for type_name, results in results_glmsingle.items():
    result_name = type_names[type_name]
    print(result_name)
    result_path = output_path / result_name
    result_path.mkdir(exist_ok=True, parents=True)

    for k, v in results.items():
        if isinstance(v, np.ndarray):
            print(k, v.dtype, v.shape)
            if v.shape[0] == num_voxels:
                if len(v.shape) == 1:
                    out_img = np.zeros_like(mask_data, dtype=v.dtype if v.dtype != bool else int)
                    out_img[mask_data] = v
                    img = nib.Nifti1Image(out_img, mask_image.affine)
                    nib.save(img, result_path / f'{result_name}_{k}.nii.gz')

        else:
            print(k, type(v))

onoff
onoffR2 float64 (190862,)
meanvol float32 (190862,)
betasmd float32 (190862, 1)
fithrf
FitHRFR2 float32 (190862, 20)
FitHRFR2run float32 (1526896, 20)
HRFindex int64 (190862,)
HRFindexrun int64 (1526896,)
R2 float32 (190862,)
R2run float32 (190862, 8)
betasmd float32 (190862, 400)
meanvol float32 (190862,)
fithrf_GLMdenoise
HRFindex int64 (190862,)
HRFindexrun int64 (1526896,)
glmbadness float32 (190862, 11)
pcvoxels bool (190862,)
pcnum <class 'int'>
xvaltrend float32 (11,)
noisepool bool (190862,)
pcregressors <class 'list'>
betasmd float32 (190862, 400)
R2 float32 (190862,)
R2run float32 (190862, 8)
meanvol float32 (190862,)
fithrf_GLMdenoise_RR
HRFindex int64 (190862,)
HRFindexrun int64 (1526896,)
glmbadness float32 (190862, 11)
pcvoxels bool (190862,)
pcnum <class 'int'>
xvaltrend float32 (11,)
noisepool bool (190862,)
pcregressors <class 'list'>
betasmd float32 (190862, 400)
R2 float32 (190862,)
R2run float32 (190862, 8)
rrbadness float32 (190862, 20)
FRACvalue float32 (190